### 기본설정

In [1]:
#!pip install selenium 
import selenium
print(f"selenium version : {selenium.__version__}")

selenium version : 4.0.0


In [23]:
from selenium import webdriver
import requests
import pandas as pd
import os
import time
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup

In [15]:
#드라이버 열기
driver_path = "chromedriver.exe"
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--start-maximized') #브라우저 최대화된 상태로 실행
driver = webdriver.Chrome(driver_path, options=chrome_options)

C:\Users\위재성\AppData\Local\Temp/ipykernel_22480/324532307.py:5: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(driver_path, options=chrome_options)


In [84]:
#데이터셋
title=[]
genre =[]
story=[]
age=[]
info=[]
tag =[]
tag2 =[]
image=[]
review=[]
sex = []
rate=[]
people=[]

### 크롤링 🔥🔥🔥🔥🔥🔥

1. 크롬 창 최소화 하지말고 띄워놓기
2. 뭔가 잘 안 되면 time.sleep() 안의 숫자 늘려보기 -> 저한테 물어보세요ㅎㅎ
3. 가끔 광고창 뜨는데 그냥 닫기 누르면 됨

In [17]:
name = pd.read_csv('name.csv')

In [85]:
cur = 1

for nm in name.iloc[573:721,0]: 
    
    #제목으로 검색하기
    url = 'https://laftel.net/finder'
    driver.get(url)    
    driver.find_element(By.XPATH,'//*[@id="root"]/div[1]/div[1]/div/div/div[3]/div/div[2]').click()
    driver.find_element(By.XPATH,'//*[@id="root"]/div[1]/div[1]/div/div/div[3]/div/div[1]/div/input').send_keys(nm, Keys.ENTER)
    time.sleep(1)
    driver.find_element(By.XPATH,'//*[@id="root"]/div[1]/div[2]/div/div[3]/div[1]/a[1]/picture').click()
    time.sleep(1)
    print('현재 ',cur,end=" ")
    
    current = driver.current_url
    

    #제목
    tit = driver.find_element(By.XPATH,'//*[@id="root"]/div[1]/div[2]/section/div/div[3]/div[1]/h1')
    tit_t = tit.text
    title.append(tit_t)

    #장르
    try:
        gen = driver.find_element(By.XPATH,'//*[@id="root"]/div[1]/div[2]/section/div/div[3]/div[2]/div[1]')
        genre.append(gen.text)

    except NoSuchElementException:
        genre.append(0)

    #제작정보 ->추가 전처리 필요함 (세 개 다 있는 경우, 하나만 있는 경우 )
    try:
        mi=driver.find_element(By.XPATH,'//*[@id="root"]/div[1]/div[2]/div[2]/aside/section[1]/div')
        info.append(mi.text)
    except NoSuchElementException:
        info.append(0)

    #태그
    try:
        tg = driver.find_element(By.XPATH,'//*[@id="root"]/div[1]/div[2]/div[2]/aside/section[2]')
        tag.append(tg.text)
    except NoSuchElementException:
        tag.append(0)

    #수상경력/ 태그2 -> 수상경력있는 애들은 태그가 뒤로 밀림. -> 태그 전처리 할때 필요
    try:
        tg2 = driver.find_element(By.XPATH,'//*[@id="root"]/div[1]/div[2]/div[2]/aside/section[3]')
        tag2.append(tg2.text)
    except NoSuchElementException:
        tag2.append(0)

    #줄거리
    try: #더보기 눌러야할 경우
        driver.find_element(By.XPATH,'//*[@id="root"]/div[1]/div[2]/div[2]/aside/div[2]/section/div/div/div').click()
        time.sleep(0.5)
        sto = driver.find_element(By.XPATH,'//*[@id="item-portal"]/div[2]/div/p')
        story.append(sto.text)
        driver.get(url) #x버튼 클릭 안돼서 일단 대안으로
    except NoSuchElementException: #안 눌러도 되는데 있는 경우
        try:
            sto = driver.find_element(By.XPATH,'//*[@id="root"]/div[1]/div[2]/div[2]/aside/div[2]/section/div/div')
            story.append(sto.text)
        except NoSuchElementException: #줄거리 아예 없는 경우
            story.append(0)

    ######################### 이미지  #########################
    driver.get(current)

    #에피소드 갯수
    try:
        epis = driver.find_element(By.XPATH,'//*[@id="root"]/div[1]/div[2]/div[2]/main/section/div/div[1]/a[1]')
        epi = epis.text.split('(')[1].split(')')[0]
    except:
        pass
     

    #스크롤 내려서 모든 에피소드 불러오기 
    body = driver.find_element(By.XPATH,'/html/body')
    while True:
        body.send_keys(Keys.PAGE_DOWN) 
        time.sleep(1)
        try:
            suc = driver.find_element(By.XPATH,'//*[@id="root"]/div[1]/div[2]/div[2]/main/section/div/div[2]/ul/li['+str(epi)+']/a/div[1]/img').get_attribute('src')
            break
        except NoSuchElementException:
            pass

    #이미지
    for i in range(1,int(epi)+1):
        try:
            img = driver.find_element(By.XPATH,'//*[@id="root"]/div[1]/div[2]/div[2]/main/section/div/div[2]/ul/li['+str(i)+']/a/div[1]/img').get_attribute('src')
            image.append([tit_t,img])
        except NoSuchElementException:
            print('에피소드',i,'없음')


    ######################### 리뷰  #########################
    driver.find_element(By.XPATH,'//*[@id="root"]/div[1]/div[2]/div[2]/main/section/div/div[1]/a[2]').send_keys(Keys.ENTER)

    #리뷰 성비
    man =  driver.find_element(By.XPATH,'//*[@id="root"]/div[1]/div[2]/div[2]/main/section/div/div[2]/div[1]/div[2]/div[2]/div[2]/div[1]').text.split('%')[0]
    sex.append(man)

    #평점 수
    rn = driver.find_element(By.XPATH,'//*[@id="root"]/div[1]/div[2]/div[2]/main/section/div/div[2]/div[1]/div[1]').text.split('(')[1].split('명')[0].replace(',','')
    people.append(rn)

    #평점
    rat = driver.find_element(By.XPATH,'//*[@id="root"]/div[1]/div[2]/section/div/div[3]/div[2]/div[2]').text.split(' ')[1]
    rate.append(rat)

    #리뷰 수
    try:
        rev = driver.find_element(By.XPATH,'//*[@id="root"]/div[1]/div[2]/div[2]/main/section/div/div[1]/a[2]').text.split('(')[1].split(')')[0].replace(',','')
    except:
        rev = 0

    #스크롤 내려서 모든 리뷰 불러오기
    last_height = driver.execute_script("return document.body.scrollHeight")        
    while True:
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(1)                                                
        new_height = driver.execute_script("return document.body.scrollHeight")
        if new_height == last_height:                                                
            break

        last_height = new_height

    #리뷰
    try:
        for k in range(1,int(rev)+1):
            #리뷰
            re = driver.find_element(By.XPATH,'//*[@id="root"]/div[1]/div[2]/div[2]/main/section/div/div[2]/div[3]/div[3]/div['+str(k)+']/p')
            
            #닉네임
            nn = driver.find_element(By.XPATH,'//*[@id="root"]/div[1]/div[2]/div[2]/main/section/div/div[2]/div[3]//div[3]/div['+str(k)+']/div[1]/div[1]')
            nnk = nn.text.split('(')[0]
            
            review.append([tit_t,re.text,nnk])
            
    except:
        pass

    print('끝')
    cur +=1

현재  1 끝
현재  2 끝
현재  3 끝
현재  4 끝
현재  5 끝
현재  6 끝
현재  7 끝
현재  8 끝
현재  9 끝
현재  10 끝
현재  11 끝
현재  12 끝
현재  13 끝
현재  14 끝
현재  15 끝
현재  16 끝
현재  17 끝
현재  18 끝
현재  19 끝
현재  20 끝
현재  21 끝
현재  22 끝
현재  23 끝
현재  24 끝
현재  25 끝
현재  26 끝
현재  27 끝
현재  28 끝
현재  29 끝
현재  30 끝
현재  31 끝
현재  32 끝
현재  33 끝
현재  34 끝
현재  35 끝
현재  36 끝
현재  37 끝
현재  38 끝
현재  39 끝
현재  40 끝
현재  41 끝
현재  42 끝
현재  43 끝
현재  44 끝
현재  45 끝
현재  46 끝
현재  47 끝
현재  48 끝
현재  49 끝
현재  50 끝
현재  51 끝
현재  52 끝
현재  53 끝
현재  54 끝
현재  55 끝
현재  56 끝
현재  57 끝
현재  58 끝
현재  59 끝
현재  60 끝
현재  61 끝
현재  62 끝
현재  63 끝
현재  64 끝
현재  65 끝
현재  66 끝
현재  67 끝
현재  68 끝
현재  69 끝
현재  70 끝
현재  71 끝
현재  72 끝
현재  73 끝
현재  74 끝
현재  75 끝
현재  76 끝
현재  77 끝
현재  78 끝
현재  79 끝
현재  80 끝
현재  81 끝
현재  82 끝
현재  83 끝
현재  84 끝
현재  85 끝
현재  86 끝
현재  87 끝
현재  88 끝
현재  89 끝
현재  90 끝
현재  91 끝
현재  92 끝
현재  93 끝
현재  94 끝
현재  95 끝
현재  96 끝
현재  97 끝
현재  98 끝
현재  99 끝
현재  100 끝
현재  101 끝
현재  102 끝
현재  103 끝
현재  104 끝
현재  105 끝
현재  106 끝
현재  107 끝
현재  108 끝
현재  109 끝
현재  110 끝
현재  111 

In [87]:
result = pd.DataFrame({'제목' : title, '장르' : genre, '줄거리' : story, '제작정보' : info, '태그' : tag, '태그2':tag2,'성비':sex,'평점':rate, '평점수':people})
result

,제목,장르,줄거리,제작정보,태그,태그2,성비,평점,평점수
0,(자막) 나소흑전기 : 첫만남편,액션 / 판타지,요정과 인간이 공존하는 환상적인 세계가 열린다! 숲속의 집을 잃고 홀로 떠돌던 검은...,출시 : 2019년,태그\n#마법#요괴 및 괴물#판타지#액션,0,11,4.8,581
1,(더빙) 페어리 테일 9기,판타지 / 액션,성령계가 무사히 안정을 되찾자 페어리 테일 멤버들도 모두 평온한 일상으로 돌아온다....,제작 : A-1Pictures,태그\n#우정#성장#배틀#마법#판타지#액션#모험#만화 원작,0,43,4.5,1869
2,한다 군 - 판권 부활,일상 / 개그,"유명 서예가 한다 세이메이의 아들로서, 고등학생임에도 불구하고 서예의 길을 걷고 있...",제작 : 디오미디어\n출시 : 2016년 3분기,태그\n#우정#성장#발랄가볍#감동#교복#미소년#일상#개그#학교#섬#농촌,0,42,4.7,887
3,유녀전기 -Saga of Tanya the Evil Movie-,판타지 / 액션,최전선에서 유녀는 비웃는다...,제작 : NUT\n출시 : 2019년 1분기,태그\n#마법#전쟁#서양풍#환생#무거움#먼치킨#판타지#액션#상상의 장소,0,83,4.7,800
4,(더빙) 페어리 테일 7기,판타지 / 액션,대마투 연무에 이은 최후의 결전이 시작된다!!\n점점 다가오는 어두운 미래! 과연 ...,제작 : A-1Pictures,태그\n#우정#성장#배틀#마법#판타지#액션#모험#만화 원작,0,41,4.5,1872
...,...,...,...,...,...,...,...,...,...
143,달의 요정 세일러 문 R,판타지 / 액션,어둠의 왕국과의 싸움이 끝나고 세일러 요정들은 자신이 세일러 요정이라는 사실을 잊고...,제작 : 토에이 애니메이션\n각본 : 토미타 스케히로\n그림 : 타케우치 나오코\n...,태그\n#미소녀#마법소녀#싸우는 미소녀#교복#판타지#액션#개그#로맨스#학교,0,23,4.2,1376
144,D4DJ First Mix,음악 / 아이돌,"그 광경을, 우리는 잊지 못한다.'\n'해피 어라운드'가 입버릇인 아이모토 린쿠.\...",제작 : SANZIGEN\n출시 : 2020년 4분기|2021년 1분기,태그\n#미소녀#발랄가볍#음악#아이돌#학교#공연장,0,74,4.6,650
145,걸즈 앤 판처,액션,니시즈미 미호는 오아라이 여학원으로 전학 온 여고생이다. 집에서 나와 혼자 기숙사에...,제작 : 액타스\n각본 : 요시노 히로유키\n그림 : 요시다 레이코\n출시 : 20...,태그\n#우정#성장#미소녀#싸우는 미소녀#귀여움#액션#학교#오리지널 애니메이션 작품,0,93,4.0,2183
146,(더빙) 오소마츠 6쌍둥이 3기,개그 / 성인,자신의 집에서 낯선 6쌍둥이 무리를 목격한 오소마츠. 오소마츠는 그들을 주인공으로 ...,제작 : 스튜디오 피에로\n그림 : 아카츠카 후지오\n출시 : 2020년 4분기|2...,태그\n#가족#충격의 1화#발랄가볍#감동#쌍둥이#개그#성인#학교#마을,0,21,4.7,450


In [122]:
pd.concat([id361_574,result]).to_csv('361-720.csv', index=False, encoding='utf-8-sig')

In [124]:
id361_574

,제목,장르,줄거리,제작정보,태그,태그2,성비,평점,평점수
0,Fate/stay night,판타지 / 액션,일본의 어느 지방도시 '후유키 시'에서는 수십 년에 한 번 '성배 전쟁'이 벌어진다...,제작 : 스튜디오 딘\n각본 : 사토 타쿠야\n출시 : 2006년,수상 이력\n라프텔 역대 애니 인기 랭킹 100위,태그\n#성장#배틀#마법#서양풍#무거움#철학#먼치킨#판타지#액션#마을#게임 원작,44,4.5,2449
1,(더빙) 일하는 세포,판타지 / 액션,"[매주 일요일 밤 11시, 2화씩 업데이트 됩니다]\n이것은 나의 몸 이야기. 나의...",제작 : 데이비드 프로덕션\n각본 : 카키하라 유코\n그림 : 시미즈 아카네\n출시...,태그\n#지식전달#판타지#액션#일상#개그,0,83,4.6,1788
2,Re : 제로부터 시작하는 이세계 생활 - 빙결의 인연,판타지 / 액션,친룡왕국 루그니카에 있는 엘리오르 대삼림. 그곳은 인간의 간섭을 거부하는 '빙결의 ...,제작 : 화이트폭스\n그림 : 오츠카 신이치로\n출시 : 2019년 4분기,태그\n#마법#이세계#히어로#이능력#루프물#판타지#액션#모험#SF#소설 원작,0,27,4.3,5645
3,XXX HOLiC 2기,판타지 / 로맨스,‘요괴’가 보이는 고교생 와타누키는 어느 날 소원을 들어주는 가게의 여주인인 이치하...,제작 : 프로덕션 I.G\n각본 : 오오카와 나나세\n출시 : 2008년,태그\n#마법#철학#몽환#판타지#로맨스#SF#미스터리#상점,관련 테마\n2018년 2월 라프텔 추가 애니 목록,84,4.5,2213
4,(무삭제) 소드 아트 온라인 앨리시제이션 War of Underworld part 1,액션 / 모험,"키리토, 유지오, 앨리스.\n두 명의 수검사와 한 명의 정합기사가 최고 사제 어드미...",제작 : A-1Pictures\n그림 : 카와하라 레키\n출시 : 2019년 4분기,태그\n#이세계#미소녀#게임#전사#먼치킨#액션#모험#SF#하렘#상상의 장소#이 라이...,관련 테마\n소드 아트 온라인 시리즈 완전정복,77,4.7,1558
...,...,...,...,...,...,...,...,...,...
207,아르테,시대물 / 드라마,무대는 16세기 초 이탈리아 피렌체 회화와 조각 등의 예술이 번창한 르네상스의 발상...,제작 : 세븐아크스\n각본 : 요시다 레이코\n그림 : 오쿠보 케이\n출시 : 20...,태그\n#서양풍#예술#시대물#만화 원작#드라마,0,87,3.8,9317
208,바보걸,일상 / 개그,"귀엽게 생긴 여학생, '하나바타케 요시코'는 실은 굉장한 바보였다! 바보의, 바보에...",제작 : 디오미디어\n각본 : 아오시마 타카시\n출시 : 2017년 3분기,태그\n#교복#일상#개그#로맨스#성인#학교#회사#만화 원작,관련 테마\n2017년 3분기 완결 애니 몰아보자! 반값세일!,81,4.3,3352
209,공의 경계 3장 통각잔류,판타지 / 액션,"1998년 7월, 신체의 여러 곳이 뒤틀린 시체가 발견되는 도저히 인간의 짓이라고 ...",제작 : 유포테이블\n출시 : 2008년 1분기,태그\n#철학#판타지#액션#성인#추리#미스터리,0,28,4.4,2332
210,극장판 XXX 홀릭 : 한여름 밤의 꿈,판타지 / 로맨스,어떤 소원이든 그에 상응하는 대가만 지불하면 다 해결해 주는 마녀 이치하라 유코.\...,제작 : 프로덕션 I.G\n각본 : 사쿠라이 요시키\n출시 : 2005년,태그\n#마법#철학#몽환#판타지#로맨스#SF#미스터리#상점,관련 테마\n8월 라프텔에 추가된 애니 목록 1탄,53,4.4,2811


In [129]:
image_361_720 = image_361_574 +image
len(image_361_720)
pd.DataFrame(image_361_720).to_csv('image_361-720.csv', index=False, encoding = 'utf-8-sig')

In [134]:
len(review_361_574+review) - 368

38467

In [133]:

pd.DataFrame(review_361_574+review).to_csv('review_361-720.csv', index=False, encoding = 'utf-8-sig')

In [64]:
#이미지 저장
import urllib.request
urllib.request.urlretrieve(img_src,"C:/Users/bounc/Desktop/애니 주분/"+'실험'+'.jpg') #파일 url, 저장위치 및 파일이름

('C:/Users/bounc/Desktop/애니 주분/실험.jpg',
 <http.client.HTTPMessage at 0x25163c87ee0>)